## Hyperparam

In [1]:
block_size = 150 + 4 +1
batch_size = 128
n_layer = 8 
n_head = 8
n_embd = 256
max_epochs = 10

# test_every_iteration = 20000 ## we test every epoch

stop_token = '$'
split_token = '&'

## Import

In [2]:
# set up logging
import logging
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO,
)

# from flattenV3Advanced import flattenData

# make deterministic
from minGPT.mingpt.utils import set_seed
set_seed(42)

import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.cuda.current_device()
from torch.utils.data.dataloader import DataLoader
from torchvision import datasets, models, transforms


import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from datetime import datetime
import math
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from torchtext.data.metrics import bleu_score

from minGPT.mingpt.model import GPT, GPTConfig
# from minGPT.mingpt.trainer import Trainer, TrainerConfig
# from minGPT.mingpt.utils import sample

from sklearn.metrics import mean_squared_error

from tqdm import tqdm
import random
import pandas as pd
import string
import re
import os
import time

import logging

logger = logging.getLogger(__name__)

from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from rdkit.Chem import AllChem, Descriptors
from rdkit import Chem
from rdkit.rdBase import BlockLogs
from rdkit.Chem import PandasTools
block = BlockLogs() ## not sure we want to block all but rdkit complain when wrong smiles are sent...

import sqlite3

## dataset path
data_path = '/home/teleport/perso/smiles_clean.db'
EXPERIMENT = '20210105_DF_gpt_smiles_pcba_first'
experiment_path = "/opt/data/train_dir/" + EXPERIMENT + "/"
if not os.path.exists(experiment_path):
    os.mkdir(experiment_path)

%matplotlib inline

01/06/2021 20:27:20 - INFO - rdkit -   Enabling RDKit 2020.09.1 jupyter extensions


## Trains and tensorboard

In [3]:
now = datetime.now()
writer = SummaryWriter(experiment_path + '/' + now.strftime("%Y%m%d-%H%M%S") + "/")
from trains import Task
import trains
task = Task.init(project_name='gpt_smiles', task_name=EXPERIMENT)

TRAINS Task: overwriting (reusing) task id=c1a099fcff034127a4bc24580ce9e4b0
TRAINS results page: http://172.22.4.157:8080/projects/8f2ac9400a46477683280746b293e09f/experiments/c1a099fcff034127a4bc24580ce9e4b0/output/log


## Dataset

In [4]:
sqliteConnection = sqlite3.connect(data_path)
cursor = sqliteConnection.cursor()
print("Database created and Successfully Connected to SQLite")

Database created and Successfully Connected to SQLite


In [5]:
cursor.execute('SELECT COUNT() FROM pcba_train')
n_train = cursor.fetchone()
n_train

(6260429,)

In [6]:
cursor.execute('SELECT COUNT() FROM pcba_test')
n_test = cursor.fetchone()
n_test

(585833,)

In [7]:
cursor.execute('SELECT * FROM pcba_train LIMIT 10')
x = cursor.fetchall()
pd.DataFrame(x)

2021-01-06 20:27:22,872 - trains.Repository Detection - WARNING - Can't get branch information for git repo in /home/teleport/.conda/envs/pytorch_smiles/lib/python3.8/site-packages


,0,1,2,3
0,CC(=O)Nc1ccc(C(=O)OCC(=O)NC(=O)NC(C)(C)C)cc1,PCBA.1030,0,1
1,C[N+]1(C)[C@H]2CC[C@@H]1CC(OC(=O)C(CO)c1ccccc1)C2,PCBA.1030,0,2
2,Cc1noc(NS(=O)(=O)c2ccc(N)cc2)c1C,PCBA.1030,0,3
3,Cc1ccc(NCCC(=O)c2cccs2)cc1,PCBA.1030,1,4
4,Oc1ccccc1CNCc1ccccc1O,PCBA.1030,0,5
5,C/C(CCN1CCCCc2nc(C)c(C)cc21)=N\O[C@H](C)c1cn([...,PCBA.1030,0,6
6,CCOC(=O)c1c(NC(=O)c2cc(C)oc2C)sc(C(=O)Nc2ccc(C...,PCBA.1030,0,7
7,C=CCOc1cc(OC)c(OC)c(OC)c1,PCBA.1030,0,8
8,COc1ccc(CC(=O)NCc2nnc(SCC(=O)Nc3nccs3)n2C)cc1,PCBA.1030,0,9
9,Cc1ccccc1NC(=O)Cn1nnc2c(cnn2-c2ccc(F)cc2)c1=O,PCBA.1030,0,10


In [8]:
cursor.execute('SELECT DISTINCT variable FROM pcba_train')
tasks = cursor.fetchall()
tasks = [i[0] for i in tasks]
tasks

2021-01-06 20:27:22,897 - trains.Repository Detection - WARNING - Can't get commit information for git repo in /home/teleport/.conda/envs/pytorch_smiles/lib/python3.8/site-packages
2021-01-06 20:27:22,908 - trains.Repository Detection - WARNING - Can't get root information for git repo in /home/teleport/.conda/envs/pytorch_smiles/lib/python3.8/site-packages
2021-01-06 20:27:22,918 - trains.Repository Detection - WARNING - Can't get status information for git repo in /home/teleport/.conda/envs/pytorch_smiles/lib/python3.8/site-packages
2021-01-06 20:27:22,939 - trains.Repository Detection - WARNING - Can't get diff information for git repo in /home/teleport/.conda/envs/pytorch_smiles/lib/python3.8/site-packages
2021-01-06 20:27:22,949 - trains.Repository Detection - WARNING - Can't get modified information for git repo in /home/teleport/.conda/envs/pytorch_smiles/lib/python3.8/site-packages
TRAINS new package available: UPGRADE to v0.17.2 is recommended!
Release Notes:
Trains is now Cle

['PCBA.1030',
 'PCBA.1458',
 'PCBA.1460',
 'PCBA.2546',
 'PCBA.2551',
 'PCBA.485297',
 'PCBA.485313',
 'PCBA.485364',
 'PCBA.504332',
 'PCBA.504333',
 'PCBA.504339',
 'PCBA.504444',
 'PCBA.504467',
 'PCBA.588342',
 'PCBA.624296',
 'PCBA.624297',
 'PCBA.624417',
 'PCBA.651965',
 'PCBA.652104',
 'PCBA.686970',
 'PCBA.686978',
 'PCBA.686979',
 'PCBA.720504']

In [9]:
if (sqliteConnection):
    sqliteConnection.close()

## word list

In [10]:
word_list = ['C','c','N','n','O','o','H','B','r','l','I','F','S','s','P','.','[',']','(',')','-','+','=','@','/','\\','#','$','&']
word_list += list(string.digits)
word_list += tasks
stoi = { ch:i for i,ch in enumerate(word_list) }
itos = { i:ch for i,ch in enumerate(word_list) }

## Helper functions for checking the smiles and tokenize them, detokenize them

In [11]:
len('asdpajsdpsd'[:10])

10

In [12]:
# def tokenize_idx(data, idx, stoi, block_size = None, stop_token = '$', split_token = '&'):
#     return tokenize_smiles(data['smiles'][idx],data['MolFormula'][idx],data['logp'][idx], stoi, block_size=block_size, stop_token = '$', split_token = '&')

def tokenize_smiles(smiles, task, results, stoi, block_size = None, stop_token = '$', split_token = '&'):
    chunk = []
    chunk.append(task)
    for i in split_token + str(results) + split_token + smiles + stop_token:
        chunk.append(i)
    if block_size is not None:
        while len(chunk) < block_size:
            chunk.append(stop_token)
        if len(chunk) > block_size:
            chunk = chunk[:block_size]
    data = [stoi[s] for s in chunk]
    return data

def y_to_completion(y, itos):
    return ''.join([itos[int(i)] for i in y])
    
    
def check_model_output(model, task, results, stop_token = '$', split_token = '&', batch_size = 4,temperature=0.9):
    completions = []
    for i, j  in zip(task, results):        
        data = [stoi[s] for s in [i , split_token , j , split_token]]
        x = [torch.tensor(data, dtype=torch.long).unsqueeze(0).to(device)]
        y = sample_clean(model, x, steps = block_size, temperature=temperature, sample=True, top_k=5, itos = itos, stop_token = '$', batch_size = batch_size)
        for idx in range(y.size(0)):    
            completion = y_to_completion(y[idx], itos)
            completions.append(completion)
    df = completion_to_pandas(completions)
            
    return df, completions
    
def completion_to_pandas(completion):
    smiles = []
    task = []
    result = []
    MolFormulaOutput = []
    for i in completion:
#         print(i)
        i = i.split(split_token)
        task.append(i[0])
        result.append(i[1])
        smile = i[2].split(stop_token)[0]
        smiles.append(smile)
        mol = Chem.MolFromSmiles(smile)
        if mol is None:
            MolFormulaOutput.append('fail')
        else:
            MolFormulaOutput.append(CalcMolFormula(mol))
    df = pd.DataFrame(list(zip(task,result, smiles, MolFormulaOutput)), 
               columns =['task','result', 'smiles', 'MolFormulaOutput']) 
    PandasTools.AddMoleculeColumnToFrame(df,smilesCol='smiles')
    return(df)

def check_df(df):
    df2 = df[df['MolFormulaOutput'] != 'fail']
    smiles_failure_ratio = (len(df) - len(df2))/len(df)
    return {'smiles_failure_ratio':smiles_failure_ratio}


# def add_mol(writer, smiles, global_step=None, size=(300, 300)):
#     """
#     Adds a molecule to the images section of Tensorboard.
#     """
#     img_transform = transforms.Compose([
#             transforms.ToTensor()
#         ])
#     mol = Chem.MolFromSmiles(smiles)
#     image = Chem.Draw.MolToImage(mol, size=size)
#     writer.add_image(smiles, img_transform(image), global_step) 


In [13]:
smiles = 'CC(=O)Nc1ccc(C(=O)OCC(=O)NC(=O)NC(C)(C)C)cc1'
task = 'PCBA.1030'
result = 0
df = tokenize_smiles(smiles, task, result, stoi, block_size)
# df

In [14]:
completion = 'truc&1&CN(C)c1ccc(CNC(=O)CN2CCC(C)(CN3CCN(C)CC3)CC2)cc1$$$$'
check_df(completion_to_pandas([completion]))

{'smiles_failure_ratio': 0.0}

## Dataloader

In [15]:
class SmilesDataset(Dataset):

    def __init__(self, table, n, id_string, block_size, stoi, itos):
        
        self.table = table
        self.n = n
        self.id_string = id_string
        self.stoi = stoi
        self.itos = itos
        self.block_size = block_size
        self.vocab_size = len(self.stoi)
        
    
    def __len__(self):
        return self.n

    def __getitem__(self, idx,verbose=False):
        try:
            sqliteConnection = sqlite3.connect(data_path)
            cursor = sqliteConnection.cursor()
            sqlite_select_Query = "SELECT * FROM " + self.table + "  WHERE " + self.id_string + " = " + str(idx+1)
            cursor.execute(sqlite_select_Query)
            record = cursor.fetchone()
            if (sqliteConnection):
                sqliteConnection.close()
#             print(record[5])
            smiles = record[0]
            task = record[1]
            result = record[2]
            converted_chunk = tokenize_smiles(smiles, task, result, self.stoi, self.block_size)
        except:
#             print('error in data loader: idx = ' + str(idx))
#             try:
#                 print(record)
#             except:
#                 print('could not even print record')
            return self.__getitem__(random.randint(0, self.n))
        x = torch.tensor(converted_chunk[:-1], dtype=torch.long)
        y = torch.tensor(converted_chunk[1:], dtype=torch.long)
        return x, y
            
train_dataset = SmilesDataset('pcba_train', n_train[0], 'id', block_size, stoi, itos) 
test_dataset = SmilesDataset('pcba_test', n_test[0], 'id', block_size, stoi, itos) 

df = train_dataset.__getitem__(idx = 10,verbose=True)
df

(tensor([39, 28, 29, 28,  0,  0,  0,  2, 30,  0,  0,  0, 18,  2, 18,  0, 19,  0,
          0,  1, 31,  1,  1,  1, 18,  4,  0, 19,  1, 18,  4,  0, 19,  1, 31, 19,
          0,  0, 30, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27]),
 tensor([28, 29, 28,  0,  0,  0,  2, 30,  0,  0,  0, 18,  2, 18,  0, 19,  0,  0,
          1, 31,  1,  1,  1, 18,  4,  0, 19,  1, 18,  4,  0, 19,  1, 31, 19,  0,
          0, 30, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27,
         27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2

In [16]:
# sqliteConnection = sqlite3.connect(data_path)
# cursor = sqliteConnection.cursor()
# sqlite_select_Query = "SELECT * FROM " + 'smiles_train' + "  WHERE " + 'id' + " = " + str(61678207+1)
# cursor.execute(sqlite_select_Query)
# record = cursor.fetchone()
# print(record)
# if (sqliteConnection):
#     sqliteConnection.close()
# #             print(record[5])
# smiles = record[1]
# molecular_formula = record[5]
# logp = record[3]
# # converted_chunk = tokenize_smiles(smiles, molecular_formula, logp, self.stoi, self.block_size)

In [17]:
# sqliteConnection = sqlite3.connect('/home/teleport/perso/smiles.db')
# cursor = sqliteConnection.cursor()
# sqlite_select_Query = "SELECT * FROM " + 'smiles_test' + "  WHERE " + 'id' + " = " + str(1)
# cursor.execute(sqlite_select_Query)
# record = cursor.fetchone()
# print(record)

## Model declaration

In [18]:
mconf = GPTConfig(train_dataset.vocab_size, train_dataset.block_size,
                  n_layer=n_layer, n_head=n_head, n_embd=n_embd)
model = GPT(mconf)
# model.load_state_dict(torch.load(experiment_path + 'model_0.pt'))

01/06/2021 20:27:23 - INFO - minGPT.mingpt.model -   number of parameters: 6.390016e+06


## Sample function

In [19]:
def top_k_logits(logits, k):
    v, ix = torch.topk(logits, k)
    out = logits.clone()
    out[out < v[:, [-1]]] = -float('Inf')
    return out

@torch.no_grad()
def sample_clean(model, xs, steps, temperature=1.0, sample=False, top_k=None, itos = None, stop_token = '$',batch_size = 4):
#     block_size = model.get_block_size()
    model.eval()
    xs = sorted(xs, key = lambda x: x.size(1))
    ## add the batch size
    xs = [torch.cat(batch_size*[x]) for x in xs]
    
    ## start by making them the same length
    x = xs[0]
    for i in range(1,len(xs)): # nothing to do if length 1
        while x.size(1) < xs[i].size(1):
            x_cond = x if x.size(1) <= block_size else x[:, -block_size:] # crop context if needed
            logits, _ = model(x_cond)
            # pluck the logits at the final step and scale by temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop probabilities to only the top k options
            if top_k is not None:
                logits = top_k_logits(logits, top_k)
            # apply softmax to convert to probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution or take the most likely
            if sample:
                ix = torch.multinomial(probs, num_samples=1)
            else:
                _, ix = torch.topk(probs, k=1, dim=-1)
            # append to the sequence and continue
            x = torch.cat((x, ix), dim=1)
        x= torch.cat((x,xs[i]), dim=0)
    
    
    ## now we can keep going...
    keep_going = True
    while keep_going:
#         print(x.size(1))
        x_cond = x if x.size(1) <= block_size else x[:, -block_size:] # crop context if needed
        logits, _ = model(x_cond)
        # pluck the logits at the final step and scale by temperature
        logits = logits[:, -1, :] / temperature
        # optionally crop probabilities to only the top k options
        if top_k is not None:
            logits = top_k_logits(logits, top_k)
        # apply softmax to convert to probabilities
        probs = F.softmax(logits, dim=-1)
        # sample from the distribution or take the most likely
        if sample:
            ix = torch.multinomial(probs, num_samples=1)
        else:
            _, ix = torch.topk(probs, k=1, dim=-1)
        # append to the sequence and continue
        x = torch.cat((x, ix), dim=1)
        if x.size(1) == block_size:
            keep_going = False
        if itos is not None:
            sequences_over = 0
            for i in range(x.size(0)):                
                if itos[int(ix[i])] == stop_token:
                    sequences_over += 1
            if sequences_over == x.size(0):
                keep_going = False
    return x



## Training loop

In [20]:
class TrainerConfig:
    # optimization parameters
    max_epochs = 10
    batch_size = 64
    learning_rate = 3e-4
    betas = (0.9, 0.95)
    grad_norm_clip = 1.0
    weight_decay = 0.1 # only applied on matmul weights
    # learning rate decay params: linear warmup followed by cosine decay to 10% of original
    lr_decay = False
    warmup_tokens = 375e6 # these two numbers come from the GPT-3 paper, but may not be good defaults elsewhere
    final_tokens = 260e9 # (at what point we reach 10% of original LR)
    # checkpoint settings
    ckpt_path = None
    num_workers = 0 # for DataLoader
#     test_every_iteration = 50000

    def __init__(self, **kwargs):
        for k,v in kwargs.items():
            setattr(self, k, v)

class Trainer:

    def __init__(self, model, train_dataset, test_dataset, config):
        self.model = model
        self.train_dataset = train_dataset
        self.test_dataset = test_dataset
        self.config = config

        # take over whatever gpus are on the system
        self.device = 'cpu'
        if torch.cuda.is_available():
            self.device = torch.cuda.current_device()
            self.model = torch.nn.DataParallel(self.model).to(self.device)

    def save_checkpoint(self):
        # DataParallel wrappers keep raw model object in .module attribute
        raw_model = self.model.module if hasattr(self.model, "module") else self.model
        logger.info("saving %s", self.config.ckpt_path)
        torch.save(raw_model.state_dict(), self.config.ckpt_path + 'model.pt')

    def train(self, writer):
        model, config = self.model, self.config
        raw_model = model.module if hasattr(self.model, "module") else model
        optimizer = raw_model.configure_optimizers(config)

        def run_epoch(split, epoch):
            is_train = split == 'train'
            model.train(is_train)
            data = self.train_dataset if is_train else self.test_dataset
            loader = DataLoader(data, shuffle=True, pin_memory=True,
                                batch_size=config.batch_size,
                                num_workers=config.num_workers)

            losses = []
            pbar = tqdm(enumerate(loader), total=len(loader)) if is_train else enumerate(loader)
            for it, (x, y) in pbar:

                # place data on the correct device
                x = x.to(self.device)
                y = y.to(self.device)

                # forward the model
                with torch.set_grad_enabled(is_train):
                    logits, loss = model(x, y)
                    loss = loss.mean() # collapse all losses if they are scattered on multiple gpus
                    losses.append(loss.item())

                if is_train:

                    # backprop and update the parameters
                    model.zero_grad()
                    loss.backward()
                    torch.nn.utils.clip_grad_norm_(model.parameters(), config.grad_norm_clip)
                    optimizer.step()

                    # decay the learning rate based on our progress
                    if config.lr_decay:
                        self.tokens += (y >= 0).sum() # number of tokens processed this step (i.e. label is not -100)
                        if self.tokens < config.warmup_tokens:
                            # linear warmup
                            lr_mult = float(self.tokens) / float(max(1, config.warmup_tokens))
                        else:
                            # cosine learning rate decay
                            progress = float(self.tokens - config.warmup_tokens) / float(max(1, config.final_tokens - config.warmup_tokens))
                            lr_mult = max(0.1, 0.5 * (1.0 + math.cos(math.pi * progress)))
                        lr = config.learning_rate * lr_mult
                        for param_group in optimizer.param_groups:
                            param_group['lr'] = lr
                    else:
                        lr = config.learning_rate

                    # report progress
                    pbar.set_description(f"epoch {epoch+1} iter {it}: train loss {loss.item():.5f}. lr {lr:e}")
                                        

            if not is_train:
                test_loss = float(np.mean(losses))
                logger.info("test loss: %f", test_loss)
                return test_loss
            else:
                train_loss = float(np.mean(losses))
                return train_loss

        best_loss = float('inf')
        self.tokens = 0 # counter used for learning rate decay
        for epoch in range(config.max_epochs):

            train_loss = run_epoch('train', epoch)
            x = (epoch+1) * len(self.train_dataset)# + it * self.config.batch_size
            writer.add_scalar('loss/train_loss',train_loss,x)
            if self.test_dataset is not None:
                test_loss = run_epoch('test', epoch)
                
                
                writer.add_scalar('loss/test_loss',test_loss,x)
                tasks = self.config.tasks
                results = self.config.results
                df, completions = check_model_output(model, tasks, results, batch_size = 16)  
                results = check_df(df)
                for i in ['smiles_failure_ratio']:
                    writer.add_scalar('results/'+i,results[i],x)
            
                self.save_checkpoint()


## Training

In [21]:
results = []
for i in ('0' * len(tasks))+('1' * len(tasks)):
    results.append(i)
# results

In [22]:
# df, completions = check_model_output(model, tasks*2, results, batch_size = 2)  

In [ ]:

# # initialize a trainer instance and kick off training
tconf = TrainerConfig(max_epochs=max_epochs, batch_size=batch_size, learning_rate=6e-4,
                      lr_decay=True, warmup_tokens=512*20, final_tokens=200*len(train_dataset)*block_size,
                      num_workers=2, ckpt_path = experiment_path,
                      tasks = tasks * 2, results = results) ## 10000 * 128 = 1M approx
trainer = Trainer(model, train_dataset, test_dataset, tconf)
trainer.train(writer)

epoch 1 iter 962: train loss 0.22068. lr 6.000000e-04:   2%|▏         | 963/48910 [02:58<2:30:44,  5.30it/s]

TRAINS Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


epoch 1 iter 1758: train loss 0.19245. lr 6.000000e-04:   4%|▎         | 1759/48910 [05:29<2:29:23,  5.26it/s]

## Test the model

In [ ]:
df, completions = check_model_output(model, tasks*2, results, batch_size = 2)  

In [ ]:
df